In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('consumption.csv')

In [3]:
def get_monthi(n):
    begin=48*31*(n-1)+1
    end=48*31*n
    if n==1:
        begin=1
    if n>1:
        end-=3*48
    if n>2:
        begin-=3*48
    if n>3:
        end-=48
    if n>4:
        begin-=48
    if n>5:
        end-=48
    if n>6:
        begin-=48
    if n>8:
        end-=48
    if n>9:
        begin-=48
    if n>10:
        end-=48
    if n>11:
        begin-=48
    return begin,end

In [ ]:
pli=True #print log info
NaN_t=1200 #NaN threshold (entire month has about 1400 datapoint)->we want atleast 1/7th of a month in this case
totSSEnd=0
nSSE=0
totSSEm=0 #total SSE mean
nSSEm=0
totSSEn=0 #total SSE naive

# loop over all meter ids
for i in range(0,df.shape[0]):
    #load row
    meter=df.iloc[i]
    fmf=False #first month found -> this variable is needed because naive drift needs 2 data points
    
    # loop over all months
    for m in range (1,13):
        ind_b,ind_e=get_monthi(m)               #get index of beginning and end of month
        month=meter[ind_b:ind_e]

        # Check if months has numeric values        
        n_NaN=month.isnull().sum() #Number of NaN's
        if n_NaN<NaN_t:
            mean_month=month.mean()
            if m<12:
                #make guesses
                naive_guess=mean_month
                mean_guess=meter[1:ind_e].mean() #mean over all data up until now
                #load next month
                next_ind_b,next_ind_e=get_monthi(m+1) 
                next_month=meter[next_ind_b:next_ind_e]
                mnm=next_month.mean()         #mean next month
                #caluclate error
                #mean
                SSEm=(mnm-mean_guess)**2
                totSSEm+=SSEm
                nSSEm+=1
                #niave
                SSEn=(mnm-naive_guess)**2
                totSSEn+=SSEn
                #naive drift needs 2 months worth of data and thus need to wait for the first month to be found
                if fmf:
                    ndrift=2*mean_month-last_month
                    #naive drift error
                    SSEnd=(mnm-ndrift)**2 
                    totSSEnd+=SSEnd
                    nSSE+=1
                    
                if pli:
                    print("i=%i m=%i mean SSE=%f   naive SSE=%f" % (i, m, SSEm, SSEn))
            else:
                fmf=True
            
            last_month=mean_month
        else:
            begin,end = get_monthi(m)
            
        
        #if pli:     
        #    print("Current month is %i" % m)
        #    print("Amount of NaN found %i" %n_NaN)

In [16]:
print("Average SSE mean method: %f" %(totSSEm/nSSEm))
print("Average SSE naive method: %f" %(totSSEn/nSSEm))

Average SSE mean method: 0.006989
Average SSE naive method: 0.003388
